In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.append('/data/home/jdavey/notebooks/fastai/mi')
from data.loader import *
from nn.lin import *
from nn.relu import *
from nn.softmaxCrossEntropy import *

In [3]:
import math
import torch

## data_loader

In [4]:
avaliable_urls()

MNIST_URL http://deeplearning.net/data/mnist/mnist.pkl


In [5]:
def normalize(x, m, s): return (x-m)/s
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
n,m = x_train.shape
c = y_train.max()+1
n, m, int(c)

(50000, 784, 10)

In [7]:
class Model():
    def __init__(self, n_in, nh, n_out):
        self.layers = [Lin(n_in,nh),
                       Relu(),
                       Lin(nh,n_out)]
        self.loss = SoftmaxCrossEntropy()
        
#     def __setattr__(self,k,v):
#         if not k.startswith("_"): self.layers[k] = v
#         super().__setattr__(k,v)
        
    def __repr__(self): 
        repr = "Model(" + "\n"
        for i, l in enumerate (self.layers):
            repr+= f"(layer_{i}):" + str(l) + "\n"
        repr+=")"
        return repr
    
    def parameters(self):
        for l in self.layers:
            for p in l.parameters(): yield p
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def forward(self, x):
        for l in self.layers: x = l(x)
        return self.loss.softmax_forward(x)
    
    def backward(self, y_train):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()
    
    def accuracy(self, pred, yb): return (torch.argmax(pred, dim=1)==yb).float().mean()

In [8]:
model = Model(m,50,int(c))
model

Model(
(layer_0):Linear(in_features=784, out_features=50)
(layer_1):Relu()
(layer_2):Linear(in_features=50, out_features=10)
)

In [9]:
[o.shape for o in model.parameters()]

[torch.Size([784, 50]),
 torch.Size([50]),
 torch.Size([50, 10]),
 torch.Size([10])]

In [10]:
loss = model(x_train, y_train); loss

tensor(-0.1224)

In [11]:
model.backward(y_train)

In [12]:
pred = model.forward(x_train)
model.accuracy(pred, y_train)

tensor(0.1467)

In [13]:
bs = 10000    # batch size
lr = 0.01   # learning rate
epochs = 20# how many epochs to train for

In [14]:
x_train.shape[0]//bs

5

In [15]:
for epoch in range(epochs):
    for i in range((n-1)//bs + 1):
        start_i = i*bs
        end_i = start_i+bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        loss = model(xb, yb)
        model.backward(yb)
        
#         for l in model.layers:
#             if hasattr(l, 'w'):
#                 l.w-=l.w.g*lr
#                 l.b-=l.b.g*lr
                
        for parameter in model.parameters():
            parameter-=lr*parameter.g
            
        if i % 5 == 0:
            pred = model.forward(x_valid)
            print(model.accuracy(pred, y_valid))

tensor(0.1649)
tensor(0.2896)
tensor(0.4120)
tensor(0.5057)
tensor(0.5670)
tensor(0.6120)
tensor(0.6401)
tensor(0.6575)
tensor(0.6744)
tensor(0.6883)
tensor(0.7049)
tensor(0.7183)
tensor(0.7305)
tensor(0.7403)
tensor(0.7530)
tensor(0.7620)
tensor(0.7710)
tensor(0.7773)
tensor(0.7840)
tensor(0.7898)
